# 7.3 항공 사진 내 선인장 식별 경진대회 베이스라인 모델
- [항공 사진 내 선인장 식별 경진대회 링크](https://www.kaggle.com/c/aerial-cactus-identification)
- [베이스라인 코드 참고 링크](https://www.kaggle.com/bonhart/simple-cnn-on-pytorch-for-beginers)

In [1]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

## 7.3.1 시드 값 고정 및 GPU 장비 설정

### 시드 값 고정

In [3]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드 값 고정
seed = 10

os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### GPU 장비 설정

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

## 7.3.2 데이터 준비하기

### 훈련 데이터, 검증 데이터 분리

In [6]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=10)

In [7]:
print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

훈련 데이터 개수: 15750
검증 데이터 개수: 1750


### 데이터 세트 정의

In [8]:
import cv2 # OpenCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir = './', transform=None):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출 
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터 세트 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 
    def __getitem__(self, idx):
        img_path = self.img_dir + self.df.iloc[idx, 0] # 이미지 파일 경로 
        image = cv2.imread(img_path) # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1] # 이미지 레이블(타깃 값)
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image)
        return image, label

### 데이터 세트 및 데이터 로더 생성

In [9]:
from torchvision import transforms # 이미지 변환을 위한 모듈

transform = transforms.ToTensor()

In [10]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

In [11]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [12]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=32, 
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, 
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g)

## 7.3.3 모델 생성

In [13]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의 
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        # 첫 번째 합성곱 계층 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                               kernel_size=3, padding=2) 
        # 두 번째 합성곱 계층 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, 
                               kernel_size=3, padding=2) 
        # 최대 풀링 계층 
        self.max_pool = nn.MaxPool2d(kernel_size=2) 
        # 평균 풀링 계층 
        self.avg_pool = nn.AvgPool2d(kernel_size=2) 
        # 전결합 계층 
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
    # 순전파 출력 정의 
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x

In [14]:
# class Model(nn.Module):
#     def __init__(self):
#         super().__init__() 
#         # 첫 번째 합성곱, 최대 풀링 계층
#         self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, 
#                                               out_channels=32, 
#                                               kernel_size=3, 
#                                               padding=2),
#                                     nn.ReLU(),
#                                     nn.MaxPool2d(kernel_size=2))
#         # 두 번째 합성곱, 최대 풀링 계층
#         self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, 
#                                               out_channels=64, 
#                                               kernel_size=3, 
#                                               padding=2),
#                                     nn.ReLU(),
#                                     nn.MaxPool2d(kernel_size=2))
#         # 평균 풀링 계층
#         self.avg_pool = nn.AvgPool2d(kernel_size=2) 
#         # 전결합 계층
#         self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
#     # 순전파 출력 정의
#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.avg_pool(x)
#         x = x.view(-1, 64 * 4 * 4) # 평탄화
#         x = self.fc(x)
#         return x

In [15]:
model = Model().to(device)

model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

## 7.3.4 모델 훈련

### 손실 함수와 옵티마이저 설정

In [16]:
# 손실함수
criterion = nn.CrossEntropyLoss()

In [17]:
# 옵티마이저
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

### 모델 훈련

In [18]:
import math

math.ceil(len(train) / 32)

493

In [19]:
len(loader_train)

493

In [20]:
epochs = 10 # 총 에폭

# 총 에폭만큼 훈련
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    # 미니배치 크기만큼 데이터를 추출하는 작업을 '반복 횟수'만큼 반복 
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
        epoch_loss += loss.item() # 현재 배치에서의 손실 추가
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')    

에폭 [1/10] - 손실값: 0.5154
에폭 [2/10] - 손실값: 0.3285
에폭 [3/10] - 손실값: 0.2359
에폭 [4/10] - 손실값: 0.1958
에폭 [5/10] - 손실값: 0.1723
에폭 [6/10] - 손실값: 0.1554
에폭 [7/10] - 손실값: 0.1465
에폭 [8/10] - 손실값: 0.1348
에폭 [9/10] - 손실값: 0.1234
에폭 [10/10] - 손실값: 0.1210


## 7.3.5 모델 성능 검증

In [21]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 예측 확률값과 실제값을 담을 리스트 초기화
true_list = []
preds_list = []

model.eval() # 모델을 평가 상태로 설정 

with torch.no_grad(): # 기울기 계산 비활성 -
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)   
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률값  
        true = labels.cpu() # 실제값 
        # 예측 확률값과 실제값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)
        
    # 검증 데이터 ROC AUC 점수 출력 
    print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')    

검증 데이터 ROC AUC : 0.9918


In [22]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, 
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g)

In [23]:
model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측 값 저장용 변수 초기화

with torch.no_grad(): # 기울기 계산 비활성
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃 값이 1일 확률(예측 값)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # preds에 preds_part 이어붙이기
        preds.extend(preds_part)

### 결과 제출

In [24]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [25]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')